# Vanilla RNN
`Author: YUAN Yanzhe`

- This notebook contains:
  - RNN Language Model from scratch
  - RNN Language Model using PyTorch
- Dataset: Jaychou's Lyrics



## RNN LM from Scratch

In [21]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import d2lzh_pytorch as d2dl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import random

In [22]:
# Obtain Data
def load_data_jay_lyrics():
    with open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_dict, char_list, vocab_size) = load_data_jay_lyrics()

# Load Data
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    num_examples = len(corpus_indices) // num_steps
    num_batch = num_examples // batch_size
    example_list = list(range(num_examples))
    random.shuffle(example_list)
    
    # obtain data in one batch
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(num_batch):
        i = i * batch_size
        batch_indices = example_list[i:i+batch_size]
        X = [_data(j*num_steps) for j in batch_indices]
        Y = [_data(j*num_steps+1) for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device), torch.tensor(Y,dtype=torch.float32,device=device)

def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    batch_num = len(corpus_indices) // batch_size
    # make a matrix over (batch_size, batch_num)
    indices = corpus_indices[0:batch_num*batch_size].view(batch_size,batch_num)
    # dived batch_num by num_steps to get consecutive examples
    batch_num_example = batch_num // num_steps  
    for i in range(batch_num_example):
        i = i * num_steps
        X = corpus_indices[:,i:i+num_steps]
        Y = corpus_indices[:,i+1:i+num_steps+1]
        yield X,Y
    
# Define Model
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def init_rnn_state(batch_size, num_hidden, device):
    state = torch.zeros(batch_size, num_hidden, device=device)
    # store the inital state into a tuple
    return (state,)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),dtype=torch.float32,device=device)
        return nn.Parameter(ts,requires_grad=True)
    
    W_ih = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    W_ho = _one((num_hiddens,num_outputs))
    b_h = nn.Parameter(torch.zeros(num_hiddens,device=device),requires_grad=True)
    b_o = nn.Parameter(torch.zeros(num_outputs,device=device),requires_grad=True)
    return nn.ParameterList([W_ih,W_hh,W_ho,b_h,b_o])

def rnn(inputs, state, params):
    W_ih,W_hh,W_ho,b_h,b_o = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X,W_ih) + torch.matmul(H,W_hh) + b_h)
        Y = torch.matmul(H,W_ho) + b_o
        outputs.append(Y)
    return outputs, (H,)

def grad_clipping(params,clipping_threshold,device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > clipping_threshold:
        for param in params:
            param.grad.data *= (clipping_threshold / norm)


In [23]:
# Train Model
def predict(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size,\
                device, char_list, char_dict):
    state = init_rnn_state(1,num_hiddens,device)
    #print(prefix[0])
    output = [char_dict[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X = d2dl.to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state) = rnn(X,state,params)
        if t < len(prefix) - 1:
            output.append(char_dict[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(1).item()))
    return ''.join([char_list[i] for i in output])
    
    
def train_and_predict(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device,\
                     corpus_indices, char_list, char_dict, is_random, num_epochs,\
                     num_steps, lr, clipping_theta, batch_size, pred_period, pred_len,\
                     prefixes, data_iter_random, data_iter_consecutive, grad_clipping):
    if is_random:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    
    params = get_params()
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random:
            state = init_rnn_state(batch_size,num_hiddens,device)
        l_sum, n = 0.0,0
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device=None)
        for X,Y in data_iter:
            if is_random:
                state = init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
            # computation graph
            inputs = d2dl.to_onehot(X,vocab_size)
            (outputs,state) = rnn(inputs,state,params)
            outputs = torch.cat(outputs,dim=0)
            y = torch.transpose(Y,0,1).contiguous().view(-1)
            l = loss(outputs,y.long())
        
            # set gradient to zero manually
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
        
            l.backward()
            grad_clipping(params,clipping_theta,device)
            d2dl.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
    
        if (epoch+1) % pred_period == 0:
            print('epoch: %d, perplexity: %f' %(epoch+1,math.exp(l_sum/n)))
            for prefix in prefixes:
                print('-',predict(prefix, pred_len, rnn, params, init_rnn_state,\
                              num_hiddens, vocab_size, device, char_list, char_dict))
        
        
        
            
        

In [24]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, char_list,
                      char_dict, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes,data_iter_random,data_iter_consecutive,grad_clipping)

epoch: 50, perplexity: 67.967254
- 分开 我不要再想 我不能再不 我不能再想 我不能再不 我不能再想 我不能再不 我不能再想 我不能再不 我
- 不分开  我不要再不 我不要我想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch: 100, perplexity: 10.108797
- 分开 我想想好生 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 我知道好生你
- 不分开吗 我已你再想 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch: 150, perplexity: 2.956918
- 分开 有不想好起 我唱往再的 我有能有  我有 这不跟久了吧? 折一枝杨柳 你在那有 在小村外的溪边河口
- 不分开吗 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不能
epoch: 200, perplexity: 1.569537
- 分开 有蟑段不起 谁辛都美 全家怕日出 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋
- 不分开想 我不能再想 我不 我不 我不能再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch: 250, perplexity: 1.319545
- 分开 有蟑想 教拳脚武术的老板 练铁沙掌 耍杨家枪 硬底子功 过目种种 象一场梦 不敢去碰 没有梦痛 不
- 不分开期 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子B 瞎 说着三么我对妈 难说你 不颗我抬起头


## RNN LM using PyTorch

一些细节
- 目标：实现语言模型
- 使用RNN模型之前
  - 定义dataloader-采样
    - 在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。
      - 这样我们对于每个batch都得初始化，因为batch之间并不连续无法保存上下文
    - 令相邻的batch在原始序列上的位置相邻。我们可以用一个batch的最终隐藏状态来初始化下一个batch的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。
      - 这样，我们只需对每个epoch进行初始化
      - 但是为了避免梯度累积在每个batch要detach一下state
  - 将数据转换成合适的格式，包括one-hot，转换成embedding，数据采样归化batches等等，如果batch内的example不同长，还需要pack。
    - 处理不定长输入（即每个batch的num_steps不定长）
      - 用`torch.nn.utils.rnn.PackedSequence`
      - https://zhuanlan.zhihu.com/p/49486149, https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.PackedSequence.html#torch.nn.utils.rnn.PackedSequence
- 定义RNN模型
  - 数据也需要转换，包括cat，stack等
  - RNN层定义的时候需要定义input_size, hidden_size等，在调用的时候RNN层的输入是inputs, state, 输出是outputs，state。
    - inputs的size是(num_steps, batch_size, input_size)
    - state是初始化的hidden state: (num_layers * num_directions, batch, hidden_size)
    - outputs的size是(num_steps, batch_size, hidden_size)，如果是bidirection的话hidden_size要乘上bi
    - state的size是(num_layers * num_direction, batch_size, hidden_size)，如果是stacked RNN，那么第一维为num_layer。
  - RNN能伸能张：RNN模块结构的num_steps时间步和batch_size都是根据实际输入来改变的，所以我们定义的RNN模型可以作为一个sequence RNN来用，也可以作为一个RNN单位。
- 训练
  - **训练过程**
    - 定义loss(y要为longTensor)和optimizor(Adam)
    - 初始化state，model转为device
    - 对每个epoch开始循环：
      - 从data loader中提取1 batch的X和Y
        - 若为consecutive每个batch提取后与上一次的分离
        - 模型输出，计算loss
        - 梯度清零，反向传播，梯度裁剪，优化器更新
        - 计算统计量
      - 一个epoch结束，输出统计量，若满足条件，开始predict/test
  - 训练过程细节
    - 初始状态可以不定义（None）
    - d2dl.grad_clipping
- 测试
  - 根据prefix来预测pred_len个输出。
  - 上一个字符的输出作为下一个的输入。
  - RNN能伸能张：RNN模块结构的num_steps时间步和batch_size都是根据实际输入来改变的，所以我们定义的RNN模型可以作为一个sequence RNN来用，也可以作为一个RNN单位。当我们用作一个单位时（即输入num_step为1），我们可以用单位RNN的输出作为下一个的输入，这样整个就是一个sequence RNN，而state也能保持一直以来的信息。


In [23]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import d2lzh_pytorch as d2dl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import random

In [24]:
# Hyperparameter
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Obtain Data
def load_data_jay_lyrics():
    with open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_dict, char_list, vocab_size) = load_data_jay_lyrics()
input_size, hidden_size = vocab_size, 256

# Load Data: batches
def dataloader_consecutive(corpus_indices, batch_size, num_steps, device=device):
    # the same as data_iter_consecutive
    corpus_indices = torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    corpus_len = len(corpus_indices)
    num_batch = corpus_len // batch_size
    indices = corpus_indices[0:batch_size*num_batch].view(batch_size,num_batch)
    num_batch_exmp = (num_batch-1) // num_steps  # consideration of Y
    for i in range(num_batch_exmp):
        i  = i * num_steps
        X = indices[:,i:i+num_steps]
        Y = indices[:,i+1:i+1+num_steps]
        yield X, Y

# Define Model
class rnnModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(rnnModel,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.state = None  # initial hidden state can be None
        self.rnn_layer = nn.RNN(input_size,hidden_size)
        self.dense_layer = nn.Linear(hidden_size,input_size)
    def forward(self, inputs, state):
        # inputs: (batch_size, num_step)
        x = d2dl.to_onehot(inputs,self.input_size)  # x: list(num_step*(batch_size, input_size))
        y, self.state = self.rnn_layer(torch.stack(x),state)  # y: (num_step, batch_size, hidden_size)
        output = self.dense_layer(y.view(-1,y.shape[-1]))  # output: (num_step, batch_size, hidden_size)
        return output, self.state

net = rnnModel(input_size,hidden_size).to(device)
print(net)
        

rnnModel(
  (rnn_layer): RNN(1027, 256)
  (dense_layer): Linear(in_features=256, out_features=1027, bias=True)
)


In [41]:
# Predict Model
def predict_lm(prefix, pred_len, net, vocab_size, device, char_dict, char_list):
    outputs = [char_dict[prefix[0]]]
    state = None
    for i in range(pred_len+len(prefix)-1):
        # initialization
        inputs = torch.tensor([outputs[-1]],device=device).view(1,1)  # (batch_size, num_step)= (1,1)
        if state is not None:
            if isinstance(state,tuple):
                state = (state[0].to(divice), state[1].to(device))  # LSTM state: (h,c)
            else:
                state = state.to(device)
        
        (Y, state) = net(inputs, state)
        
        if i < len(prefix)-1:
            outputs.append(char_dict[prefix[i+1]])
        else: 
            outputs.append(int(Y.argmax(1).item()))  # argmax returns index
    return ''.join([char_list[item] for item in outputs])
        
#predict_lm('分', 20, net, vocab_size, device, char_dict, char_list)

# Train Model: consecutive sampling
def train_lm(model, num_hiddens, vocab_size, device, corpus_indices, char_list, char_dict,\
            num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes):
    # define loss and optimizor
    loss = nn.CrossEntropyLoss()
    optimizor = optim.Adam(model.parameters(), lr=lr)
    
    model = model.to(device)
    state = None
    
    for epoch in range(num_epochs):
        state=None
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = dataloader_consecutive(corpus_indices,batch_size,num_steps,device)
        for X, Y in data_iter:  # X:(batch_size, num_step), Y:(batch_size, num_step)
            # consecutive sampling requires detach at each batch to reduce computation on grads.
            if state is not None:
                if isinstance(state,tuple):
                    state = (state[0].detach(),state[1].detach())
                else:
                    state = state.detach()
        
            # modeling
            (outputs, state) = model(X, state)  # outputs: (num_step*batch_size,output_size)
            y = torch.transpose(Y,0,1).contiguous().view(-1)  # y: (num_step*batch_size)
            l = loss(outputs,y.long())
        
            # bp
            optimizor.zero_grad()
            l.backward()
            d2dl.grad_clipping(model.parameters(),clipping_theta,device)  
            optimizor.step()
        
            # stats
            l_sum += l.item()
            n += 1
        
        try:
            perplexity = math.exp(l_sum/n)
        except OverflowError:
            perplexity = float('inf')
            
        if (epoch+1) % pred_period == 0:  # predict after pred_period time
            print('epoch %d, perplexity %f, time %.2f sec'%(epoch+1, perplexity, \
                                                                time.time()-start))
            for prefix in prefixes:
                print('-', predict_lm(prefix, pred_len, net, vocab_size, device, \
                                          char_dict, char_list))  


In [42]:
# Main:
num_steps = 32
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_lm(net, hidden_size, vocab_size, device, corpus_indices, char_list, char_dict,\
        num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)


epoch 50, perplexity 1.011058, time 0.47 sec
- 分开的了口被废拆封 誓言太沉重泪被纵容 脸上汹涌失控 穿梭时间的画面的钟 从反方向开始移动 回到当初爱你
- 不分开不了我 难过 是因为闷了很久 是因为想了太多 是心理起了作用 你说 苦笑常常陪着你 在一起有点勉强 
epoch 100, perplexity 1.008328, time 0.54 sec
- 分开的了口 老慢再 原 用楔形文字刻下了永远 那已风化千年的誓言 一切又重演 我感到很疲倦离家乡还是很远
- 不分开不了我 娘子 你在我了多年 它一直在身边 干什么 干什么 我打开任督二脉 干什么 干什么 东亚病夫的
epoch 150, perplexity 1.008404, time 0.52 sec
- 分开 我笑常不知不  不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 
- 不分开不了我 能回到你身边 我给你的爱写在西元前 深埋在美索不达米亚平原 几十个世纪后出土发现 泥板上的字
epoch 200, perplexity 1.008087, time 0.47 sec
- 分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 说散 你想很久了吧? 我的认真败
- 不分开不了我 不知不觉 我跟了这节奏 后知后觉 后知后觉 迷迷蒙蒙 你给的梦 出现裂缝 隐隐作痛 怎么沟通
epoch 250, perplexity 1.007815, time 0.50 sec
- 分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 不想太多 我想一定是我听错弄错搞
- 不分开不了我 谁你看透 的就像是童话没有 在别人有 难熬  什么都到过得很不会呵护著你 这样 甜蜜 让我开
